# Projet Apache Spark : Analyse des données climatiques mondiales

## Objectif :
Analyser les tendances climatiques mondiales à l'aide de Spark, y compris le nettoyage des données, l'EDA et l'extraction d'informations.

## Groupe :
#### Bérenger AKODO 
#### Etienne VACHER
#### Mamoudou NDONGO

### Jeu de données :(Sur 50 ans :: 1929 - 1977)
[Global Surface Summary of the Day (GSOD) provenant de NOAA](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00516)


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
from pyspark.sql import SparkSession

# Créer ou obtenir une session Spark active
spark = SparkSession.builder \
    .appName("Meteo Analysis") \
    .config("spark.network.timeout", "14400s") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()

# Définir le chemin vers les fichiers CSV contenant les données des cyclistes
path = "./datas/50/*/*.csv"

# Charger les données CSV dans un DataFrame Spark
climat = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(path)

# Afficher les types de données (schéma) de chaque colonne dans le DataFrame
print(climat.dtypes)

[('STATION', 'bigint'), ('DATE', 'date'), ('LATITUDE', 'double'), ('LONGITUDE', 'double'), ('ELEVATION', 'double'), ('NAME', 'string'), ('TEMP', 'double'), ('TEMP_ATTRIBUTES', 'double'), ('DEWP', 'double'), ('DEWP_ATTRIBUTES', 'double'), ('SLP', 'double'), ('SLP_ATTRIBUTES', 'double'), ('STP', 'double'), ('STP_ATTRIBUTES', 'double'), ('VISIB', 'double'), ('VISIB_ATTRIBUTES', 'double'), ('WDSP', 'double'), ('WDSP_ATTRIBUTES', 'double'), ('MXSPD', 'double'), ('GUST', 'double'), ('MAX', 'double'), ('MAX_ATTRIBUTES', 'string'), ('MIN', 'double'), ('MIN_ATTRIBUTES', 'string'), ('PRCP', 'double'), ('PRCP_ATTRIBUTES', 'string'), ('SNDP', 'double'), ('FRSHTT', 'int')]


In [3]:
# Inspection du schéma et calcul des statistiques
climat.printSchema()
climat.count(), len(climat.columns)

root
 |-- STATION: long (nullable = true)
 |-- DATE: date (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- TEMP_ATTRIBUTES: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- DEWP_ATTRIBUTES: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- SLP_ATTRIBUTES: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- VISIB_ATTRIBUTES: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- WDSP_ATTRIBUTES: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- MIN: double (nullable = true)
 |-- MIN_ATTRIBUTES: string (nullable = true)
 |-- PRCP: double (nullable = true)

(4013704, 28)

In [4]:
climat.count()

4013704

In [5]:
climat.printSchema()

root
 |-- STATION: long (nullable = true)
 |-- DATE: date (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- NAME: string (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- TEMP_ATTRIBUTES: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- DEWP_ATTRIBUTES: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- SLP_ATTRIBUTES: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- STP_ATTRIBUTES: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- VISIB_ATTRIBUTES: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- WDSP_ATTRIBUTES: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: double (nullable = true)
 |-- MAX_ATTRIBUTES: string (nullable = true)
 |-- MIN: double (nullable = true)
 |-- MIN_ATTRIBUTES: string (nullable = true)
 |-- PRCP: double (nullable = true)

In [6]:
climat.select("STATION").distinct().count()

3438

In [7]:
# Identifier les colonnes avec des valeurs manquantes et les traiter
missing_values = climat.select([F.count(F.when(F.col(c).isNull(), c)) for c in climat.columns])

In [8]:
# Supprimer les lignes avec des valeurs critiques manquantes
climat_annee_delete = climat.dropna(subset=['TEMP', 'DATE'])

In [9]:
# Calculer le nombre de valeurs manquantes pour chaque colonne
missing_values = climat.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in climat.columns]
)

# Afficher chaque ligne (ici une seule ligne avec les résultats pour chaque colonne)
for row in missing_values.collect():
    print(row.asDict())

{'STATION': 0, 'DATE': 0, 'LATITUDE': 107936, 'LONGITUDE': 107936, 'ELEVATION': 108599, 'NAME': 106035, 'TEMP': 0, 'TEMP_ATTRIBUTES': 0, 'DEWP': 0, 'DEWP_ATTRIBUTES': 0, 'SLP': 0, 'SLP_ATTRIBUTES': 0, 'STP': 0, 'STP_ATTRIBUTES': 0, 'VISIB': 0, 'VISIB_ATTRIBUTES': 0, 'WDSP': 0, 'WDSP_ATTRIBUTES': 0, 'MXSPD': 0, 'GUST': 0, 'MAX': 0, 'MAX_ATTRIBUTES': 0, 'MIN': 0, 'MIN_ATTRIBUTES': 0, 'PRCP': 0, 'PRCP_ATTRIBUTES': 0, 'SNDP': 0, 'FRSHTT': 0}


In [10]:
# Supprimer les lignes avec des valeurs manquantes dans n'importe quelle colonne
drop_climat = climat.dropna()

# Compter le nombre d'enregistrements restants après nettoyage
remaining_records = drop_climat.count()

print(f"Nombre d'enregistrements après nettoyage : {remaining_records}")

Nombre d'enregistrements après nettoyage : 3904665


In [11]:
#Pour l'ensemble de notre étude, nous avons choisi d'adopter la méthode d'imputation des 
#valeurs manquantes en utilisant une valeur fixe, telle que la moyenne, la médiane, ou d'autres 
#valeurs appropriées.

# Récupérer la liste des colonnes numériques
numeric_columns = [field.name for field in climat.schema.fields if isinstance(field.dataType, (IntegerType, DoubleType))]

# Calculer la moyenne de chaque colonne numérique et remplir les valeurs manquantes
mean_values = climat.select([F.mean(c).alias(c) for c in numeric_columns]).collect()[0].asDict()

# Remplacer les valeurs manquantes pour chaque colonne par la moyenne
climat_cleaned = climat.fillna(mean_values)

# Afficher le nombre d'enregistrements après nettoyage
remaining_records = climat_cleaned.count()
print(f"Nombre d'enregistrements après nettoyage : {remaining_records}")

Nombre d'enregistrements après nettoyage : 4013704


In [12]:
def remove_unwanted_columns(climat_cleaned: DataFrame, columns_to_remove: list) -> DataFrame:
    """
    Supprime les colonnes spécifiées d'un DataFrame PySpark.
    
    :param df: DataFrame d'entrée
    :param columns_to_remove: Liste des colonnes à supprimer
    :return: DataFrame nettoyé sans les colonnes spécifiées
    """
    return climat_cleaned.drop(*columns_to_remove)

# Liste des colonnes à supprimer
columns_to_remove = [
    "TEMP_ATTRIBUTES",
    "DEWP_ATTRIBUTES",
    "SLP_ATTRIBUTES",
    "STP_ATTRIBUTES",
    "VISIB_ATTRIBUTES",
    "WDSP_ATTRIBUTES",
    "MAX_ATTRIBUTES",
    "MIN_ATTRIBUTES",
    "PRCP_ATTRIBUTES"
]

# Suppression des colonnes dans le DataFrame climat
climat_col_cleaned = remove_unwanted_columns(climat_cleaned, columns_to_remove)
#Rajout de la colonne Celcius
climat_col_cleaned = climat_col_cleaned.withColumn("TEMP_C", bround((col("TEMP") - 32) / 1.8, 1))
 
# Afficher le DataFrame avec les colonnes 'TEMP' et 'TEMP_C' arrondies
climat_col_cleaned.select("TEMP", "TEMP_C").show()
# Afficher les colonnes restantes pour vérifier
print("Colonnes restantes :", climat_col_cleaned.columns)

+----+------+
|TEMP|TEMP_C|
+----+------+
|47.2|   8.4|
|33.0|   0.6|
|26.7|  -2.9|
|26.2|  -3.2|
|31.4|  -0.3|
|27.3|  -2.6|
|20.2|  -6.6|
|18.6|  -7.4|
|25.6|  -3.6|
|30.5|  -0.8|
|21.1|  -6.1|
|31.2|  -0.4|
|33.1|   0.6|
|36.0|   2.2|
|44.8|   7.1|
|33.8|   1.0|
|35.5|   1.9|
|38.1|   3.4|
|29.5|  -1.4|
|36.7|   2.6|
+----+------+
only showing top 20 rows

Colonnes restantes : ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'TEMP', 'DEWP', 'SLP', 'STP', 'VISIB', 'WDSP', 'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP', 'SNDP', 'FRSHTT', 'TEMP_C']


In [13]:
# Transformer le jeu de données
# Ajouter de nouvelles colonnes pour l'année, le mois et le jour
climat_transformed = (
    climat_col_cleaned
    .withColumn('year', F.year(F.col('DATE')))  # Extraire l'année
    .withColumn('month', F.month(F.col('DATE')))  # Extraire le mois
    .withColumn('day', F.dayofmonth(F.col('DATE')))  # Extraire le jour
)

# Classifier les jours chauds (les données sont en degrées F°)
climat_transformed = climat_transformed.withColumn('is_hot', F.col('TEMP') > 68)

climat_transformed.show()

+-----------+----------+--------+---------+---------+--------------------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+------+----+-----+---+------+
|    STATION|      DATE|LATITUDE|LONGITUDE|ELEVATION|                NAME|TEMP|DEWP|   SLP|  STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|TEMP_C|year|month|day|is_hot|
+-----------+----------+--------+---------+---------+--------------------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+------+----+-----+---+------+
|99999914768|1952-01-01|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|47.2|43.1|1011.9|991.5|  9.3|13.8| 19.0|999.9|53.1|39.9|99.99|999.9|110000|   8.4|1952|    1|  1| false|
|99999914768|1952-01-02|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|33.0|27.9|1025.7|  4.7| 11.9| 9.0| 15.0|999.9|39.0|30.0|  0.0|999.9|     0|   0.6|1952|    1|  2| false|
|99999914768|1952-01-03|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|26.7|21.4|1024.5|  3.3|  7.1|11

In [14]:
climat_transformed.show(5)

+-----------+----------+--------+---------+---------+--------------------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+------+----+-----+---+------+
|    STATION|      DATE|LATITUDE|LONGITUDE|ELEVATION|                NAME|TEMP|DEWP|   SLP|  STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|TEMP_C|year|month|day|is_hot|
+-----------+----------+--------+---------+---------+--------------------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+------+----+-----+---+------+
|99999914768|1952-01-01|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|47.2|43.1|1011.9|991.5|  9.3|13.8| 19.0|999.9|53.1|39.9|99.99|999.9|110000|   8.4|1952|    1|  1| false|
|99999914768|1952-01-02|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|33.0|27.9|1025.7|  4.7| 11.9| 9.0| 15.0|999.9|39.0|30.0|  0.0|999.9|     0|   0.6|1952|    1|  2| false|
|99999914768|1952-01-03|43.11723|-77.67539|    164.5|FREDERICK DOUGLAS...|26.7|21.4|1024.5|  3.3|  7.1|11

In [15]:
# Compter le nombre de jours chauds (is_hot = true)
hot_days_count = climat_transformed.filter(F.col("is_hot") == True).count()

# Afficher le résultat
print(f"Nombre total de jours classés comme 'chauds' : {hot_days_count}")

Nombre total de jours classés comme 'chauds' : 1110333


In [16]:
# Nous avons supprimé les colonnes qui nous semblaient inutiles, géré les valeurs manquantes, 
#ajouté de nouvelles colonnes, et créé une colonne supplémentaire intitulée is_hot et une autre colonne pour
# transformer les degrées Fahrenheit en degrées Celsius

In [17]:
# Calculer les moyennes annuelles et autres statistiques
# avg_temp_by_year = df_transformed.groupBy('year').avg('temperature')
#avg_temp_by_year.show()

In [18]:
# Définir le chemin vers les fichiers CSV contenant les données des cyclistes
climat_2020 = climat_transformed.filter(F.col('year') == 2020)

avg_temp_2020 = climat_2020.agg({"TEMP": "mean", "TEMP_C": "mean"}).withColumnRenamed("avg(TEMP)", "Mean_Temperature_Fahrenheit") \
                                                                   .withColumnRenamed("avg(TEMP_C)", "Mean_Temperature_Celcius")

# Afficher les résultats
avg_temp_2020.show()


+---------------------------+------------------------+
|Mean_Temperature_Fahrenheit|Mean_Temperature_Celcius|
+---------------------------+------------------------+
|                       NULL|                    NULL|
+---------------------------+------------------------+



In [19]:

# Calculer la température moyenne par station et obtenir les autres informations
avg_temp_by_station = climat_transformed.groupBy('STATION', 'NAME', 'LATITUDE', 'LONGITUDE').agg({"TEMP": "mean", "TEMP_C": "mean"}).withColumnRenamed("avg(TEMP)", "Mean_Temperature_Fahrenheit") \
                                                                                                                                    .withColumnRenamed("avg(TEMP_C)", "Mean_Temperature_Celcius")
# Trier par température moyenne (ordre décroissant) et prendre les 5 premières stations
top_5_stations = avg_temp_by_station.orderBy(col('Mean_Temperature_Fahrenheit').desc()).limit(5)
 
# Afficher les résultats
top_5_stations.show()

+-----------+--------------------+----------+----------+---------------------------+------------------------+
|    STATION|                NAME|  LATITUDE| LONGITUDE|Mean_Temperature_Fahrenheit|Mean_Temperature_Celcius|
+-----------+--------------------+----------+----------+---------------------------+------------------------+
|74901099999|LAGUNA ARMY AIR F...|     32.85|    -114.4|          94.21052631578948|       34.56578947368422|
|74926499999|         KYI EIK, BM|    21.633|    96.033|          90.30555555555556|      32.388888888888886|
|48053099999|        MEIKTILA, BM|20.8333333|95.8333333|          89.62926829268297|      32.012195121951216|
|48047099999|        MYINGYAN, BM|21.4666666|95.3833333|          88.69666666666666|      31.500000000000007|
|42705099999|         PURULIA, IN|23.3333333|86.4166666|          87.34387755102044|       30.74897959183672|
+-----------+--------------------+----------+----------+---------------------------+------------------------+



In [20]:
 
# Calculer la somme des précipitations mondiales par année
# Calculer la température moyenne par station et obtenir les autres informations
total_precipitation = climat_transformed.groupBy('year').agg({"PRCP": "sum"}).withColumnRenamed("sum(PRCP)", "total_precipitation")
                                                                                                                                
# Trier par température moyenne (ordre décroissant) et prendre les 5 premières stations
precip_by_year = total_precipitation.orderBy(col('year').desc()).limit(50)

# Afficher l'évolution des précipitations par année
precip_by_year.orderBy('year').show()

+----+-------------------+
|year|total_precipitation|
+----+-------------------+
|1929|  94335.60999999994|
|1930| 283451.13999999996|
|1931| 268856.46000000037|
|1932|  266578.7199999999|
|1933| 396390.39999999997|
|1934| 464106.10999999975|
|1935| 495380.50999999995|
|1936|  949142.6699999995|
|1937| 2001261.3600000031|
|1938|  873775.1600000003|
|1939|          927931.89|
|1940|  1189441.350000002|
|1941| 1685390.5900000026|
|1942|  3192311.150000006|
|1943| 6350615.9100000225|
|1944|  9241402.879999947|
|1945|  9786895.800000032|
|1946|   4536058.27000001|
|1947|  4735983.610000031|
|1948|  9710215.549999936|
+----+-------------------+
only showing top 20 rows



In [21]:
# Enregistrer comme vue SQL temporaire et exécuter des requêtes
climat_transformed.createOrReplaceTempView('climate')
# spark.sql('SELECT ...')

In [22]:
# Calculer la température moyenne par année
coldest_month = spark.sql('''
    SELECT year, AVG(TEMP) AS Mean_Temperature_Fahrenheit, AVG(TEMP_C) AS Mean_Temperature_Celcius
    FROM climate
    GROUP BY year
    ORDER BY Mean_Temperature_Fahrenheit ASC
    LIMIT 1
''')

# Afficher le mois le plus froid et sa température moyenne
coldest_month.show()

+----+---------------------------+------------------------+
|year|Mean_Temperature_Fahrenheit|Mean_Temperature_Celcius|
+----+---------------------------+------------------------+
|1937|          44.56013802362421|       6.978064607303943|
+----+---------------------------+------------------------+



In [23]:
# Trouver la station avec le plus grand nombre d'enregistrements
most_records_station = spark.sql('''
    SELECT STATION, NAME, COUNT(*) AS Nombre_Enregistrement
    FROM climate
    GROUP BY STATION, NAME
    ORDER BY Nombre_Enregistrement DESC
    LIMIT 1
''')

# Afficher le résultat
most_records_station.show(truncate=False)

+-----------+----------------+---------------------+
|STATION    |NAME            |Nombre_Enregistrement|
+-----------+----------------+---------------------+
|72286023119|MARCH AFB, CA US|7271                 |
+-----------+----------------+---------------------+



In [24]:
#Pour trouver le mois le plus froid et la température moyenne :
    #SELECT year, AVG(TEMP) AS Mean_Temperature_Fahrenheit, AVG(TEMP_C) AS Mean_Temperature_Celcius
    #FROM climate
    #GROUP BY year
    #ORDER BY Mean_Temperature_Fahrenheit ASC
    #LIMIT 1
#Pour trouver la station avec le plus grand nombre d'enregistrements :
    #SELECT STATION, NAME, COUNT(*) AS Nombre_Enregistrement
    #FROM climate
    #GROUP BY STATION, NAME
    #ORDER BY Nombre_Enregistrement DESC
    #LIMIT 1

In [25]:
# Exporter les données vers Pandas et créer des visualisations
# import matplotlib.pyplot as plt
# df_pandas = avg_temp_by_year.toPandas()
# plt.plot(df_pandas['year'], df_pandas['avg_temperature'])

In [26]:
# Convertir le DataFrame PySpark en DataFrame Pandas


#climat_transformed_small = climat_transformed.limit(20)  # Exemple : limiter à 1000 lignes
#climat_transformed_pandas = climat_transformed_small.toPandas()
climat_transformed_pandas = climat_transformed.toPandas()

# Afficher les premières lignes du DataFrame Pandas
print(climat_transformed_pandas.head())


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_com

AssertionError: Undefined error message parameter for error class: CANNOT_PARSE_DATATYPE. Parameters: {'error': 'An error occurred while calling o409.schema'}

In [ ]:
#ANNEE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Extraire l'année à partir de la colonne DATE
climat_transformed_pandas['year'] = pd.to_datetime(climat_transformed_pandas['DATE']).dt.year

# Calcul de la température moyenne par année
year_avg = climat_transformed_pandas.groupby('year')['TEMP_C'].mean().reset_index()

# Ajouter les années manquantes (par exemple, de 1950 à 1955)
all_years = pd.DataFrame({'year': range(climat_transformed_pandas['year'].min(), climat_transformed_pandas['year'].max() + 1)})
year_avg = pd.merge(all_years, year_avg, on='year', how='left')
year_avg['TEMP_C'] = year_avg['TEMP_C'].fillna(0)  # Remplace les valeurs manquantes

# Préparer les données pour la heatmap
heatmap_data = year_avg.set_index('year').T  # Transposer pour avoir les années comme colonnes

# Dessin de la heatmap
plt.figure(figsize=(8, 2))
sns.heatmap(heatmap_data, cmap="coolwarm", annot=True, cbar_kws={'label': 'Température moyenne (°C)'})
plt.title("Températures moyennes mondiales par année")
plt.xlabel("Année")
plt.ylabel("")
plt.show()

In [ ]:

# Étape 1 : Filtrer les données
# Exclure les jours sans précipitation (0 mm) et les valeurs aberrantes comme 99.99 mm
precip_data_filtered = climat_transformed_pandas[(climat_transformed_pandas['PRCP'] > 0) & 
                                                 (climat_transformed_pandas['PRCP'] < 100)]['PRCP']

# Vérification : S'assurer que les données filtrées ne sont pas vides
if precip_data_filtered.empty:
    print("Aucune donnée valide après le filtrage.")
else:
    # Étape 2 : Créer un histogramme avec KDE pour mieux comprendre la distribution
    plt.figure(figsize=(10, 6))  # Ajuster la taille du graphique
    sns.histplot(precip_data_filtered, bins=30, kde=True, color="blue", stat="density")  # Stat=“density” pour une échelle normalisée

    # Étape 3 : Ajouter des titres et des labels pour clarifier l'interprétation
    plt.title("Distribution des Précipitations (mm)", fontsize=16)  # Titre du graphique
    plt.xlabel("Précipitations (mm)", fontsize=14)  # Label de l'axe X
    plt.ylabel("Densité", fontsize=14)  # Label de l'axe Y (densité, car on utilise stat="density")

    # Étape 4 : Ajouter une grille pour une meilleure lisibilité
    plt.grid(True)

    # Afficher le graphique
    plt.show()
